In [3]:
import math
import logging
FORMAT = '[%(name)s:%(levelname)s]  %(message)s'
logging.basicConfig(level=logging.DEBUG, format=FORMAT)
logger = logging.getLogger('dbg')

def dprint(s):
    logger.debug(s)

def iprint(s):
    logger.info(s)

logger.setLevel(logging.INFO)

In [56]:
## Graphs Class for demonstration of algorithms

## Directed Edge Object
class Edge():
    def __init__(self, a, b, weight) -> None:
        self.a = a
        self.b = b
        self.w = weight

## Vertex Object
class Vertex():
    def __init__(self, v) -> None:
        self.v = v

        self.color = "white"
        self.distance = float('inf')
        self.pi = None
        self.d = 0
        self.f = 0

    ## configure type comparisons
    def __contains__(self, item):
        if isinstance(item, Vertex):
            return item.v == self.v
        return item == self.v
    def __eq__(self, item):
        if isinstance(item, Vertex):
            return item.v == self.v
        return item == self.v
    
    def __repr__(self) -> str:
        if self.pi is not None:
            return f"{self.v} parent[{self.pi.v}] C:[{self.distance}] d[{self.d}] f[{self.f}] [{self.color}]"
        else:
            return f"{self.v}  C:[{self.distance}] d[{self.d}] f[{self.f}] [{self.color}]"

class Graph():
    ## Internal Adj matrix
    
    ## Build from lists by default
    # V: List of Vertex Elements
    # E: List of Edge Elements
    def __init__(self, V: list, u: list, v: list, w: list) -> None:
        self.V = []
        for q in V:
            self.V.append(Vertex(q))
        self.E = []

        # if we have edge data
        if u is not None:
            for i in range(len(u)):
                self.E.append(Edge(u[i], v[i], w[i]))
            # construct adjacency matrix/dict
            self.build_dict()

    def load_edge_from_tuple(self, E: list) -> None:
        self.E = []
        for i in range(len(E)):
            self.E.append(Edge(E[i][0], E[i][1], E[i][2]))
        # construct adjacency matrix/dict
        self.build_dict()

    def __repr__(self) -> str:
        return str(self.Aj)

    def build_array(self):
        self.Aj = []
        ## Create the array
        self.Aj = [ [ 0 for _ in self.V] for _ in self.V]
        ## fill it in
        for e in self.E:
            u_i = self.V.index(e.a)
            v_i = self.V.index(e.b)
            self.Aj[u_i][v_i] = e.w

    def build_dict(self):
        self.Aj = dict()
        # allocate the primary dict
        for vert in self.V:
            self.Aj[vert.v] = dict()
        ## fill it in
        for edge in self.E:
            self.Aj[edge.a][edge.b] = edge.w

    def get_vertex(self, a) -> Vertex:
        return self.V[self.V.index(a)]



_V = ["s","a","b","c","d","e","f","g"]
_u = ["s","s","a","b","d","d","e","e","c"]
_v = ["a","b","d","e","e","g","d","c","f"]
_w = [1 for _ in _v]

G = Graph(_V, _u, _v, _w)
print(G)

{'s': {'a': 1, 'b': 1}, 'a': {'d': 1}, 'b': {'e': 1}, 'c': {'f': 1}, 'd': {'e': 1, 'g': 1}, 'e': {'d': 1, 'c': 1}, 'f': {}, 'g': {}}


## Graphs - Basics

A graph is a mathematical object containing a set of *vertices* or corners $V$, and a set of *edges* connecting these vertices $E$. 

**Common representations:**
- *Adjacency List* - an array $A_j$ of $|V|$ unique lists, where each index of $A_j$ corresponds to a vertex $u \in V$, and the indexed list contains all vertices $v$ where there is an edge from $u$ to $v$ or equivalently  $(u,v) \in E$

Memory: **$\Theta(|V| + |E|)$**

E.g. for $|V|$ = 5, $V$ = $[1, 2, 3, 4, 5]$, a encyclic circular graphs could be described as:

| $u$ = 1   | $u$ = 2   | $u$ = 3   | $u$ = 4   | $u$ = 5   |
| --        | --        | --        | --        | --        |          
| 2         | 3         | 4         | 5         | 1         |

- *Adjacency Matrix* - a $|V|\times|V|$ binary matrix where 1 corresponds to $E$, and 0 not.

Memory: **$\Theta(|V|^2)$**

E.g. for $|V|$ = 5, $V$ = $[1, 2, 3, 4, 5]$, a encyclic circular graphs could be described as:

| $u$ = 1   | $u$ = 2   | $u$ = 3   | $u$ = 4   | $u$ = 5   |
| --        | --        | --        | --        | --        |          
| ~         | 0         | 0         | 0         | 1         |
| 1         | ~         | 0         | 0         | 0         |
| 0         | 1         | ~         | 0         | 0         |
| 0         | 0         | 1         | ~         | 0         |
| 0         | 0         | 0         | 1         | ~         |


**Types:**
* Directed
* Undirected
* Weighted
* Unweighted

**Algorithms:**
* **Graph Search & Traversal**
    * Breadth-First Search (BFS)
    * Depth-First Search (DFS)
* **Minimum Weight Spanning Trees**
    * Kruskal's Al
    * Primm's Al
* **Shortest Path**
    * Bellman-Ford 
    * Dijkstra
    * Floyd-Warshall
* **Maximum Flow**
    * Ford-Fulkerson


## Search & Traversal

### BFS

Expands the frontier between discovered and undiscovered vertices uniformly across the breadth. BFS finds all vertices at a distance $k$ from source $s$, before discovering any that are at a distance $k+1$.

Given a graph $G=(V, E)$, and a **source** vertex $S$:
* Discover every vertex that is reachable from $S$
* Discover distance $d(v)$ from $S$ to each reachable vertext $v$
* Create a BFS tree, starting at $S$, that contains all reachable vertices

**Implementation:**
* Classify vertices into: 
    * undiscovered (white)
    * a frontier between discovered and undiscovered vertices (gray)
    *discovered vertices (black).
* Starting from the first vertex on frontier, append all undiscovered neighboring nodes 
* Proceed until all are discovered


In [57]:
# BFS Implementation

_V = ["s","a","b","c","d","e","f","g"]
_u = ["s","s","a","b","d","d","e","e","c"]
_v = ["a","b","d","e","e","g","d","c","f"]
_w = [1 for _ in _v]

G = Graph(_V, _u, _v, _w)

def init_bfs(G: Graph, s: Vertex):
    for v in G.V:
        v.color = "white"
        v.distance = float('inf')
        v.pi = None
        
    s.color = "gray"
    s.distance = 0

def bfs(G: Graph, s: Vertex):
    init_bfs(G, s)
    Q = []; Q.append(s)
    while len(Q):
        u = Q.pop(0)
        # loop through th edges belonging to u
        for v in G.Aj[u.v]:
            w = G.Aj[u.v][v]
            vv = G.get_vertex(v)
            if (vv.color == "white"):
                vv.color = "gray"
                vv.distance = u.distance + 1
                vv.pi = u
                Q.append(vv)
            u.color = "black"

bfs(G, G.get_vertex("s"))

for v in G.V:
    print(v)


s  C:[0] d[0] f[0] [black]
a parent[s] C:[1] d[0] f[0] [black]
b parent[s] C:[1] d[0] f[0] [black]
c parent[e] C:[3] d[0] f[0] [black]
d parent[a] C:[2] d[0] f[0] [black]
e parent[b] C:[2] d[0] f[0] [black]
f parent[c] C:[4] d[0] f[0] [gray]
g parent[d] C:[3] d[0] f[0] [gray]


### DFS

Choose a first undiscovered vertex and explore neighbors recursively until no unexplored neighbors are left. For each vertex $v$, once all out going edges are discovered, the algorithm must backtrack to vertex from which $v$ was discovered.

Undiscovered vertices are coloured `white`, those with fully explored neighbors `black`, and the rest `gray`.

Given $G$, compute a predecessor subgraph $G_\pi$.
* The graph comprises several depth-first-search trees
* Structural information is obtained via start and finish times of each node

In [67]:
_V = ["s","a","b","c","d","e","f","g"]
ET= [("s", "b", 1), \
    ("s", "c", 1), \
    ("a", "s", 1), \
    ("a", "b", 1), \
    ("b", "c", 1), \
    ("b", "d", 1), \
    ("c", "s", 1), \
    ("d", "a", 1), \
    ("e", "b", 1), \
    ("f", "c", 1), \
    ("f", "e", 1), \
    ("f", "g", 1), \
    ("g", "d", 1), \
    ("g", "e", 1), \
    ("g", "f", 1)]
G = Graph(_V, None, None, None)
G.load_edge_from_tuple(ET)

# DFS Implementation
time = 0
def DFS_visit(G: Graph, u: Vertex):
    global time
    time += 1
    u.d = time
    u.color = "gray"
    for v in G.Aj[u.v]:
        w = G.Aj[u.v][v]
        vv = G.get_vertex(v)
        if (vv.color == "white"):
            vv.pi = u
            DFS_visit(G, vv)
    u.color = "black"
    time += 1
    u.f = time

def DFS(G: Graph):
    global time
    time = 0
    for v in G.V:
        v.color = "white"
        v.d = 0
        v.f = 0
        v.pi = None
    
    for u in G.V:
        if u.color == "white":
            DFS_visit(G, u)
    return time


DFS(G)
for v in G.V:
    print(v)
    

s  C:[inf] d[1] f[10] [black]
a parent[d] C:[inf] d[6] f[7] [black]
b parent[s] C:[inf] d[2] f[9] [black]
c parent[b] C:[inf] d[3] f[4] [black]
d parent[b] C:[inf] d[5] f[8] [black]
e  C:[inf] d[11] f[12] [black]
f  C:[inf] d[13] f[16] [black]
g parent[f] C:[inf] d[14] f[15] [black]


### Topological Sort

A Topological sort of a directed acycled graph (DAG) is a linear ordering of its vertices such that an edge $(u, v)$ requires $u$ appear before $v$ in the ordering.

If the graph contains a cycle this is of course impossible.

In [70]:
def toposort(G: Graph):
    t = DFS(G)
    A = [-1 for i in range(t+1)]
    for v in G.V:
        A[v.f] = v
    A = list(filter(lambda a: a != -1, A))
    A.reverse()
    return A

_V = ["s","a","b","c","d","e","f","g"]
ET= [("s", "b", 1), \
    ("s", "c", 1), \
    ("b", "c", 1), \
    ("b", "d", 1), \
    ("d", "a", 1), \
    ("e", "b", 1), \
    ("f", "c", 1), \
    ("f", "e", 1), \
    ("f", "g", 1), \
    ("g", "d", 1), \
    ("g", "e", 1)]

G = Graph(_V, None, None, None)
G.load_edge_from_tuple(ET)

A = toposort(G)
for v in A:
    print(v)

f  C:[inf] d[13] f[16] [black]
g parent[f] C:[inf] d[14] f[15] [black]
e  C:[inf] d[11] f[12] [black]
s  C:[inf] d[1] f[10] [black]
b parent[s] C:[inf] d[2] f[9] [black]
d parent[b] C:[inf] d[5] f[8] [black]
a parent[d] C:[inf] d[6] f[7] [black]
c parent[b] C:[inf] d[3] f[4] [black]
